In [1]:
pip install tenacity


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [104]:
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt,wait_exponential
import urllib
import time
import numpy as np
import pandas as pd

#tenacityのライブラリに関する記事は下記
#https://zenn.dev/taroman_zenn/articles/dd0b33a3a37d1e

In [145]:
#Step1必要な情報を取得する

#goodroom上で必要な情報を検索する
#今回はペルソナに合致した下記の条件を設定｜ペルソナ：https://www.dropbox.com/scl/fi/pdo0x2sm6es6n76sek2i7/Tech0_STEP3-1_-_v1.0.pdf?rlkey=2attz3z14yyoanwm53spxk6t1&dl=0
#検索条件：東京都内、20万円以下（管理費・共益費込み）、2K以上の間取り、駅徒歩10分以内、バストイレ別、室内洗濯機置場
#取得したURL：https://www.goodrooms.jp/tokyo/search/area_list/?required_condition=is_bath_separated-has_inside_washer_place&layout=1LDK-2K-2DK-2LDK-3K-3DK-3LDK-3LDK_more&small_area_cd=1000-1001-1002-1003-1004-1005-1010-1011-1012-1013-1014-1015-1016-1020-1021-1022-1030-1031-1032-1033-1040-1041-1042&walk_minutes=10&rent_select=1&is_contain_full=0&rental_fee=-200000

#複数ページの情報をまとめて取得するリストを作成
data_samples = []

#スクレイピングするページ数を指定
#出力されたページ数52を上回るように設定する（goodroomではURLの末尾にページ数が記載されている）
max_page = 55

#goodroomで自分が調べたい地域を指定して検索し、出力したページのURL
URL = 'https://www.goodrooms.jp/tokyo/search/area_list/?required_condition=is_bath_separated-has_inside_washer_place&layout=1LDK-2K-2DK-2LDK-3K-3DK-3LDK-3LDK_more&small_area_cd=1000-1001-1002-1003-1004-1005-1010-1011-1012-1013-1014-1015-1016-1020-1021-1022-1030-1031-1032-1033-1040-1041-1042&walk_minutes=10&rent_select=1&is_contain_full=0&rental_fee=-200000&p={:d}'
goodroom_URL ='https://www.goodrooms.jp'

#試行回数3回、リトライ間の初期待機時間10秒、連続するリトライの時間を100秒にする（指数関数的に増やすが、100秒までで制限する）
#Pythonでは、デコレータの引数を|で結合することはできないため、,で指定すること
@retry(stop=stop_after_attempt(3),wait=wait_exponential(multiplier=10, min=10, max=100))
def load_page(URL):
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup


In [146]:
# 処理時間を測りたい
start = time.time()
times = []

#関数については下記を参考にすること
#enumerate()関数 https://note.nkmk.me/python-enumerate-start/
#strip()関数 https://note.nkmk.me/python-str-remove-strip/#strip


for page in range(1,max_page+1):
    before = time.time()
    #ページ情報
    soup = load_page(URL.format(page))
    #物件リストを指定
    mother = soup.find_all(class_='searchEstateList')

    #物件ごとの処理
    #goodroomは各物件ごとで情報が完結しているので、child以下の繰り返し処理は作成しなくてOK
    for child in mother:

        #建物情報
        data_home = []
        #建物名
        titles = child.find('h2', class_=lambda value: value and value.startswith('searchEstateList__title'))
        # それぞれのタイトルのテキストを取得します
        data_home.append(titles.text.strip())
        #家賃と管理費
        price_div = child.find('div', class_='searchEstateList__price')
        price = price_div.contents[0].strip()
        management_cost = price_div.find('span', class_='searchEstateList__managementCost').text.strip()
        data_home.append(price)
        data_home.append(management_cost)

        #敷金と礼金と仲介手数料
        costs = child.find_all('li', class_='searchEstateList__cost__detail')
        for cost in costs:
            value = cost.find('span').next_sibling.strip()
            data_home.append(value)

        # costs_detail = child.find_all('ul', class_='searchEstateList__cost__detail')
        # costs_detail_value = costs_detail.find('span').next_sibling.strip()
        # for cost_detail in costs_detail_value:
        #     data_home.append(costs_detail_value)

        #住所/階/間取り/面積
        spec_div = child.find('div', class_='searchEstateList__spec')
        specs = spec_div.text.split('/') # divタグのテキストを取得し、'/'で分割します
        for spec in specs:
            data_home.append(spec.strip())
        #最寄駅のアクセス
        data_home.append(child.find(class_='searchEstateList__traffic').text.replace('\n',''))

        #url
        get_url = child.find(class_='searchEstateList__wrap').get('href')
        abs_url = urllib.parse.urljoin(goodroom_URL, get_url)
        data_home.append(abs_url)


        #物件情報と部屋情報を結合する
        data_sample = data_home
        data_samples.append(data_sample)



    #1アクセスごとに1秒休む
    time.sleep(1)

    #進捗確認
    #このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'総取得件数:{len(data_home)}')
    #作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了:{complete_ratio}%')
    #作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間:{hour}時間{minute}分{second}秒\n')


# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

総取得件数:12
完了:1.818%
残り時間:0時間3分8秒

総取得件数:12
完了:3.636%
残り時間:0時間2分37秒

総取得件数:12
完了:5.455%
残り時間:0時間2分20秒

総取得件数:12
完了:7.273%
残り時間:0時間2分6秒

総取得件数:12
完了:9.091%
残り時間:0時間2分1秒

総取得件数:12
完了:10.909%
残り時間:0時間2分2秒

総取得件数:12
完了:12.727%
残り時間:0時間1分57秒

総取得件数:12
完了:14.545%
残り時間:0時間1分52秒

総取得件数:12
完了:16.364%
残り時間:0時間1分50秒

総取得件数:12
完了:18.182%
残り時間:0時間1分45秒

総取得件数:12
完了:20.0%
残り時間:0時間1分42秒

総取得件数:12
完了:21.818%
残り時間:0時間1分38秒

総取得件数:12
完了:23.636%
残り時間:0時間1分35秒

総取得件数:12
完了:25.455%
残り時間:0時間1分32秒

総取得件数:12
完了:27.273%
残り時間:0時間1分29秒

総取得件数:12
完了:29.091%
残り時間:0時間1分29秒

総取得件数:12
完了:30.909%
残り時間:0時間1分30秒

総取得件数:12
完了:32.727%
残り時間:0時間1分27秒

総取得件数:12
完了:34.545%
残り時間:0時間1分25秒

総取得件数:12
完了:36.364%
残り時間:0時間1分22秒

総取得件数:12
完了:38.182%
残り時間:0時間1分20秒

総取得件数:12
完了:40.0%
残り時間:0時間1分18秒

総取得件数:12
完了:41.818%
残り時間:0時間1分15秒

総取得件数:12
完了:43.636%
残り時間:0時間1分13秒

総取得件数:12
完了:45.455%
残り時間:0時間1分10秒

総取得件数:12
完了:47.273%
残り時間:0時間1分8秒

総取得件数:12
完了:49.091%
残り時間:0時間1分5秒

総取得件数:12
完了:50.909%
残り時間:0時間1分2秒

総取得件数:12
完了:52.727%
残り時間:0時間0分59秒

総

In [147]:
data_samples

[['TOMOS\u3000おひさまパワー',
  '154,000円',
  '管理費2,000円',
  '1ヶ月',
  '1ヶ月',
  '1.1ヶ月',
  '東京都目黒区',
  '301号室',
  '1LDK',
  '40.0㎡',
  '都立大学駅 徒歩5分(東急東横線)',
  'https://www.goodrooms.jp/tokyo/detail/001/1001302171/'],
 ['秋を楽しみに',
  '98,000円',
  '管理費10,000円',
  '1ヶ月',
  '1ヶ月',
  '1.1ヶ月',
  '東京都文京区',
  '406号室',
  '1LDK',
  '34.31㎡',
  '千石駅 徒歩8分(東京都三田線)',
  'https://www.goodrooms.jp/tokyo/detail/001/1000161593/'],
 ['TOMOS\u3000黄昏ロマン',
  '135,000円',
  '管理費なし',
  '1ヶ月',
  '1ヶ月',
  '1.1ヶ月',
  '東京都中野区',
  '201号室',
  '1LDK',
  '40.7㎡',
  '中野新橋駅 徒歩7分(東京地下鉄方南支線)',
  'https://www.goodrooms.jp/tokyo/detail/001/6144201/'],
 ['ほんの少しの異国情緒',
  '130,000円',
  '管理費8,000円',
  '1ヶ月',
  '1ヶ月',
  '1.1ヶ月',
  '東京都文京区',
  '102号室',
  '1LDK',
  '33.68㎡',
  '巣鴨駅 徒歩6分(山手線)',
  'https://www.goodrooms.jp/tokyo/detail/001/1001310843/'],
 ['ぼくらのお料理研究所',
  '145,000円',
  '管理費5,000円',
  '1ヶ月',
  '1ヶ月',
  '1.1ヶ月',
  '東京都荒川区',
  '803号室',
  '1LDK',
  '42.66㎡',
  '町屋駅 徒歩1分(東京地下鉄千代田線)',
  'https://www.goodrooms.jp/tokyo/detail/001/100

In [148]:
#Step2データを整形する

#リスト内包表記を使用し、data_samplesリストの各要素からアイテムを取得し、新しいリスト split_data_samplesへ格納する
#item[0]となっているのは、タプル内の要素が1つのリストのみであるため＝([],)となっている

# DataFrame の作成
df = pd.DataFrame(data_samples)
df.columns = ['物件名','賃料', '管理費', '敷金','礼金','仲介金','所在地','部屋の階数', '間取り', '面積', 'アクセス1', 'URL']

df

,物件名,賃料,管理費,敷金,礼金,仲介金,所在地,部屋の階数,間取り,面積,アクセス1,URL
0,TOMOS おひさまパワー,"154,000円","管理費2,000円",1ヶ月,1ヶ月,1.1ヶ月,東京都目黒区,301号室,1LDK,40.0㎡,都立大学駅 徒歩5分(東急東横線),https://www.goodrooms.jp/tokyo/detail/001/1001...
1,秋を楽しみに,"98,000円","管理費10,000円",1ヶ月,1ヶ月,1.1ヶ月,東京都文京区,406号室,1LDK,34.31㎡,千石駅 徒歩8分(東京都三田線),https://www.goodrooms.jp/tokyo/detail/001/1000...
2,TOMOS 黄昏ロマン,"135,000円",管理費なし,1ヶ月,1ヶ月,1.1ヶ月,東京都中野区,201号室,1LDK,40.7㎡,中野新橋駅 徒歩7分(東京地下鉄方南支線),https://www.goodrooms.jp/tokyo/detail/001/6144...
3,ほんの少しの異国情緒,"130,000円","管理費8,000円",1ヶ月,1ヶ月,1.1ヶ月,東京都文京区,102号室,1LDK,33.68㎡,巣鴨駅 徒歩6分(山手線),https://www.goodrooms.jp/tokyo/detail/001/1001...
4,ぼくらのお料理研究所,"145,000円","管理費5,000円",1ヶ月,1ヶ月,1.1ヶ月,東京都荒川区,803号室,1LDK,42.66㎡,町屋駅 徒歩1分(東京地下鉄千代田線),https://www.goodrooms.jp/tokyo/detail/001/1000...
...,...,...,...,...,...,...,...,...,...,...,...,...
1645,Ｃｌｏｖｅｒ Ｈｉｌｌｓ,"125,000円","管理費5,000円",1ヶ月,なし,1.1ヶ月,東京都目黒区,101号室,1LDK,38.5㎡,目黒駅 徒歩8分(山手線),https://www.goodrooms.jp/tokyo/detail/001/1001...
1646,白壁と窓の洞窟,"152,000円","管理費3,000円",なし,なし,1.1ヶ月,東京都品川区,401号室,1LDK,31.09㎡,目黒駅 徒歩5分(山手線),https://www.goodrooms.jp/tokyo/detail/001/1000...
1647,けさ何食べた？,"139,000円","管理費3,000円",1ヶ月,なし,1.1ヶ月,東京都目黒区,201号室,1LDK,37.52㎡,学芸大学駅 徒歩9分(東急東横線),https://www.goodrooms.jp/tokyo/detail/001/1001...
1648,ホワイトチョコで作る夢,"124,000円","管理費10,000円",1ヶ月,なし,1.1ヶ月,東京都大田区,103号室,1LDK,33.83㎡,馬込駅 徒歩4分(東京都浅草線),https://www.goodrooms.jp/tokyo/detail/001/1001...


In [149]:
#Step3
#出力した結果をCSVファイルに保存する
goodroom_df = df
goodroom_df.to_csv('goodroom_data.csv', index=False, encoding='utf-8-sig')